In [2]:
import pandas as pd

# Specify the file path to your dataset
file_path = 'C:\\Users\\user\\Downloads\\archive (3)\\creditcard.csv'

# Load the dataset
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
print(df.head())


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features and target variable
X = df.drop('Class', axis=1)  # Features
y = df['Class']  # Target variable

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features to standardize them
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
clf = RandomForestClassifier(n_estimators=10, random_state=42)

# Train the model
clf.fit(X_train_scaled, y_train)


RandomForestClassifier(n_estimators=10, random_state=42)

In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Make predictions
y_pred = clf.predict(X_test_scaled)

# Evaluate the model
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))
print('\nClassification Report:')
print(classification_report(y_test, y_pred))
print(f'Accuracy Score: {accuracy_score(y_test, y_pred)}')


Confusion Matrix:
[[56862     2]
 [   27    71]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.97      0.72      0.83        98

    accuracy                           1.00     56962
   macro avg       0.99      0.86      0.92     56962
weighted avg       1.00      1.00      1.00     56962

Accuracy Score: 0.9994908886626171


In [7]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
sm = SMOTE(random_state=42)

# Fit SMOTE to the training data
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train)

# Now X_train_res and y_train_res contain the oversampled data which is balanced


In [8]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(clf, X_train_res, y_train_res, cv=5)

# Print each cv score (accuracy) and average them
print(cv_scores)
print(f'cv_scores mean: {np.mean(cv_scores)}')


[0.99983513 0.99989009 0.99985711 0.9998681  0.99987909]


NameError: name 'np' is not defined

In [ ]:
# First, make sure to import numpy
import numpy as np

# Assuming cv_scores is already defined as it seems from your output
print(cv_scores)
print(f'cv_scores mean: {np.mean(cv_scores)}')


In [9]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the grid search
grid_clf = GridSearchCV(clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_clf.fit(X_train_res, y_train_res)

# Print the best parameters and best score
print(f'Best parameters: {grid_clf.best_params_}')
print(f'Best score: {grid_clf.best_score_}')


Fitting 3 folds for each of 324 candidates, totalling 972 fits


In [ ]:
from xgboost import XGBClassifier

# Initialize XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False)

# Train the model
xgb_clf.fit(X_train_res, y_train_res)

# Predict on the test data
y_pred_xgb = xgb_clf.predict(X_test_scaled)

# Evaluate the model
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


In [ ]:
import matplotlib.pyplot as plt

# Get feature importances
importances = clf.feature_importances_

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({'feature': list(X.columns), 'importance': importances})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

# Plot
feature_importance_df.plot(kind='bar', x='feature')
plt.title('Feature Importance')
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve

# Get probability estimates for the positive class
y_scores = clf.predict_proba(X_test_scaled)[:, 1]

# Calculate precision, recall, and thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_scores)

# Find threshold that maximizes a particular performance metric (e.g., F1 score)
# Code for threshold tuning goes here
